In [1]:
# import library
import pickle
import pandas as pd
import numpy as np

import xgboost as xgb
import lightgbm as lgb
import time

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="1", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)
set_session(tf.Session(config=config))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# load data
df = pd.concat([
    pd.read_csv('../data/feature_selection_positive.csv'),
    pd.read_csv('../data/decomp_pos.csv', index_col=0).drop('Subclass', axis=1)
], axis=1)

In [9]:
pd.read_csv('../data/decomp_pos.csv', index_col=0)

,Subclass,HCD35_50.0,HCD35_50.01,HCD35_50.02,HCD35_50.03,HCD35_50.04,HCD35_50.05,HCD35_50.06,HCD35_50.07,HCD35_50.08,...,NMF_0_6,NMF_1_6,NMF_2_6,NMF_3_6,NMF_4_6,FAG_0_6,FAG_1_6,FAG_2_6,FAG_3_6,FAG_4_6
0,Carbohydrates and carbohydrate conjugates,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,79.793660,4.856276,165.331777,519.419094,0.000,0.00,6.633375e+05,0.00
1,Carbohydrates and carbohydrate conjugates,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.056481,0.000000,9.334904,6.348131,180.486132,345.333060,8162.955,0.00,6.818314e+04,0.00
2,Carbohydrates and carbohydrate conjugates,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,32.521485,6.248924,165.552720,356.540224,0.000,0.00,2.684483e+05,0.00
3,Carbohydrates and carbohydrate conjugates,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,5552.914518,163.398803,0.000000,6033.707775,0.000,0.00,4.613901e+07,3176429.25
4,Carbohydrates and carbohydrate conjugates,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.536485,0.000000,3036.960868,201.525583,0.000000,3619.126583,0.000,0.00,2.466848e+07,3855051.75
5,Diterpenoids,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,9.351722,11.777570,26.851807,0.000000,293.565697,1355.871397,66508.440,303689.56,2.060121e+05,0.00
6,Diterpenoids,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.210813,0.000000,0.758852,0.049178,0.816560,32.622711,4509.470,0.00,5.164610e+03,1284.14
7,Diterpenoids,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.148922,0.000000,0.948401,0.000000,2.205358,57.408429,639.935,0.00,1.020622e+04,0.00
8,Diterpenoids,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.372544,0.381266,0.363247,0.000000,13.123361,55.070581,4126.050,10110.92,2.590266e+03,375.67
9,Diterpenoids,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.713471,0.030946,32.985334,0.066846,21.850743,289.167178,12509.360,0.00,2.670729e+05,0.00


In [3]:
# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [4]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [5]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [6]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

# append accuracy to list
t = []
t.append(clf.score(X_test, y_test))

In [7]:
# setting params
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [8]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

t.append(model.score(X_test, y_test))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [9]:
# define and fit 
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

t.append(gbm.score(X_test, y_test))

[1]	valid_0's multi_logloss: 1.30448
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.239
[3]	valid_0's multi_logloss: 1.18339
[4]	valid_0's multi_logloss: 1.1281
[5]	valid_0's multi_logloss: 1.08462
[6]	valid_0's multi_logloss: 1.0461
[7]	valid_0's multi_logloss: 1.00806
[8]	valid_0's multi_logloss: 0.976291
[9]	valid_0's multi_logloss: 0.946748
[10]	valid_0's multi_logloss: 0.91945
[11]	valid_0's multi_logloss: 0.89539
[12]	valid_0's multi_logloss: 0.873456
[13]	valid_0's multi_logloss: 0.852748
[14]	valid_0's multi_logloss: 0.83253
[15]	valid_0's multi_logloss: 0.815644
[16]	valid_0's multi_logloss: 0.797712
[17]	valid_0's multi_logloss: 0.7811
[18]	valid_0's multi_logloss: 0.765787
[19]	valid_0's multi_logloss: 0.752949
[20]	valid_0's multi_logloss: 0.740114
[21]	valid_0's multi_logloss: 0.728138
[22]	valid_0's multi_logloss: 0.716548
[23]	valid_0's multi_logloss: 0.706148
[24]	valid_0's multi_logloss: 0.696737
[25]	valid_0's multi_logloss

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [12]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)
t.append(model.evaluate(X_test, y_test_for_keras, verbose=0)[1])

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 2s 1ms/step - loss: 1.8981 - acc: 0.2425 - val_loss: 1.3891 - val_acc: 0.2470
Epoch 2/100
1340/1340 [==============================] - 0s 338us/step - loss: 1.6739 - acc: 0.2955 - val_loss: 1.3526 - val_acc: 0.2798
Epoch 3/100
1340/1340 [==============================] - 0s 337us/step - loss: 1.6679 - acc: 0.2619 - val_loss: 1.3318 - val_acc: 0.3393
Epoch 4/100
1340/1340 [==============================] - 0s 335us/step - loss: 1.5940 - acc: 0.3104 - val_loss: 1.3201 - val_acc: 0.3423
Epoch 5/100
1340/1340 [==============================] - 0s 336us/step - loss: 1.5293 - acc: 0.3164 - val_loss: 1.3218 - val_acc: 0.3363
Epoch 6/100
1340/1340 [==============================] - 0s 332us/step - loss: 1.5481 - acc: 0.3284 - val_loss: 1.3256 - val_acc: 0.3423
Epoch 7/100
1340/1340 [==============================] - 0s 335us/step - loss: 1.4388 - acc: 0.3463 - val_loss: 1.3099 - val_acc: 0.3

In [13]:
f = []
f.append(t)

In [14]:
# load data
df = pd.read_csv('../data/decomp_neg.csv', index_col=0)

In [15]:
# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [16]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [17]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [18]:
# define and fit 
clf = rf()
clf = clf.fit(X_train, y_train)

# append accuracy to list
t = []
t.append(clf.score(X_test, y_test))

In [19]:
# setting params
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [20]:
# define and fit 
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

t.append(model.score(X_test, y_test))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [21]:
# define and fit 
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

t.append(gbm.score(X_test, y_test))

[1]	valid_0's multi_logloss: 1.32513
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.2634
[3]	valid_0's multi_logloss: 1.21589
[4]	valid_0's multi_logloss: 1.17031
[5]	valid_0's multi_logloss: 1.1328
[6]	valid_0's multi_logloss: 1.09801
[7]	valid_0's multi_logloss: 1.06526
[8]	valid_0's multi_logloss: 1.0371
[9]	valid_0's multi_logloss: 1.01485
[10]	valid_0's multi_logloss: 0.990556
[11]	valid_0's multi_logloss: 0.96851
[12]	valid_0's multi_logloss: 0.947001
[13]	valid_0's multi_logloss: 0.929509
[14]	valid_0's multi_logloss: 0.901345
[15]	valid_0's multi_logloss: 0.880662
[16]	valid_0's multi_logloss: 0.86281
[17]	valid_0's multi_logloss: 0.847879
[18]	valid_0's multi_logloss: 0.833557
[19]	valid_0's multi_logloss: 0.825082
[20]	valid_0's multi_logloss: 0.815305
[21]	valid_0's multi_logloss: 0.805099
[22]	valid_0's multi_logloss: 0.797888
[23]	valid_0's multi_logloss: 0.786515
[24]	valid_0's multi_logloss: 0.780042
[25]	valid_0's multi_loglo

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [22]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)
t.append(model.evaluate(X_test, y_test_for_keras, verbose=0)[1])

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 1s 4ms/step - loss: 2.3534 - acc: 0.1823 - val_loss: 1.3775 - val_acc: 0.3404
Epoch 2/100
373/373 [==============================] - 0s 136us/step - loss: 1.9143 - acc: 0.2788 - val_loss: 1.3239 - val_acc: 0.4043
Epoch 3/100
373/373 [==============================] - 0s 131us/step - loss: 1.7048 - acc: 0.3432 - val_loss: 1.3847 - val_acc: 0.4894
Epoch 4/100
373/373 [==============================] - 0s 134us/step - loss: 1.5442 - acc: 0.3887 - val_loss: 1.4542 - val_acc: 0.5319
Epoch 5/100
373/373 [==============================] - 0s 123us/step - loss: 1.6122 - acc: 0.3324 - val_loss: 1.4717 - val_acc: 0.5319
Epoch 6/100
373/373 [==============================] - 0s 118us/step - loss: 1.6792 - acc: 0.4075 - val_loss: 1.4647 - val_acc: 0.5319
Epoch 7/100
373/373 [==============================] - 0s 122us/step - loss: 1.6042 - acc: 0.3941 - val_loss: 1.4246 - val_acc: 0.5426
Epoch 8/100


In [23]:
f.append(t)

In [28]:
round(pd.DataFrame(
    f, 
    columns=['Random Forest', 'LightGBM', 'XGBoost', 'Keras'],
    index=['Positive', 'Negative']
)*100, 2).to_csv('../result/decomp.csv')